In [2]:
import os

# Change current working directory to parent directory
# Save the original working directory
original_cwd = os.getcwd()

# Change to parent directory
os.chdir("..")


from tools import *
from dotenv import load_dotenv
from tools import *

link = 'https://www.google.com/maps/place/BAIETA+Restaurant+Saigon+Thao+Dien/@10.8029833,106.7246931,17z/data=!3m1!4b1!4m6!3m5!1s0x3175271f9bf9fd45:0x7404fcd4128a3514!8m2!3d10.802978!4d106.727268!16s%2Fg%2F11svytm6sq?entry=ttu&g_ep=EgoyMDI1MDYwMi4wIKXMDSoASAFQAw%3D%3D&hl=vi'

In [5]:
import pandas as pd

# Read the CSV file for Ho Chi Minh restaurants
df = pd.read_csv('data/tripadvisor_restaurant/Ha Noi.csv')
df.head(1).T

,0
restaurant_id,19255094
location_id,19255094
name,Hoang's Restaurant
description,Hoang's Restaurant - Vietnamese Restaurant & V...
web_url,https://www.tripadvisor.com/Restaurant_Review-...
address_obj,"{'street1': '54 Hang Buom Street', 'city': 'Ho..."
ancestors,"[{'level': 'City', 'name': 'Hoan Kiem', 'locat..."
latitude,21.036827
longitude,105.85146
timezone,Asia/Ho_Chi_Minh


In [33]:
import json

def extract_address_string(s):
    try:
        s_json = s.replace("'", '"')
        address_dict = json.loads(s_json)
        return address_dict.get('address_string')
    except Exception:
        return None

df['address_string'] = df['address_obj'].apply(extract_address_string)


In [35]:
df.head(1).T

,0
restaurant_id,19255094
location_id,19255094
name,Hoang's Restaurant
description,Hoang's Restaurant - Vietnamese Restaurant & V...
web_url,https://www.tripadvisor.com/Restaurant_Review-...
address_obj,"{'street1': '54 Hang Buom Street', 'city': 'Ho..."
ancestors,"[{'level': 'City', 'name': 'Hoan Kiem', 'locat..."
latitude,21.036827
longitude,105.85146
timezone,Asia/Ho_Chi_Minh


In [36]:
import pandas as pd

# Read the CSV file for Ho Chi Minh restaurants
# df = pd.read_csv('data/tripadvisor_restaurant/Ha Noi.csv')

# Create restaurant_string column
def get_restaurant_string(row):
    return f"{row['name']} {row['address_string']}"

df['restaurant_string'] = df.apply(get_restaurant_string, axis=1)

# Prepare output CSV file
output_path = 'data/tripadvisor_restaurant/restaurant_links.csv'
with open(output_path, mode='w', encoding='utf-8-sig', newline='') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(['restaurant_id', 'restaurant_link'])

    # Setup Selenium driver
    from selenium import webdriver
    from selenium.webdriver.common.action_chains import ActionChains
    from tools import get_restaurant_link
    driver = webdriver.Chrome(options=chrome_options)
    actions = ActionChains(driver)
    driver.get('https://www.google.com/maps/?hl=vi')

    # Iterate and get links
    for _, row in df.iterrows():
        restaurant_id = row['restaurant_id']
        restaurant_string = row['restaurant_string']
        try:
            link = get_restaurant_link(driver, f'Nhà hàng {restaurant_string}')
        except Exception:
            link = ''
        writer.writerow([restaurant_id, link])

    driver.quit()

In [37]:
import pandas as pd
import csv
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from tools import get_restaurant_link

# Read your DataFrame
# df = pd.read_csv('data/tripadvisor_restaurant/Ha Noi.csv')

def get_restaurant_string(row):
    return f"{row['name']} {row['address_string']}"

df['restaurant_string'] = df.apply(get_restaurant_string, axis=1)

output_path = 'data/tripadvisor_restaurant/restaurant_links.csv'

def crawl_link(row):
    driver = webdriver.Chrome(options=chrome_options)
    actions = ActionChains(driver)
    driver.get('https://www.google.com/maps/?hl=vi')
    restaurant_id = row['restaurant_id']
    restaurant_string = row['restaurant_string']
    try:
        link = get_restaurant_link(driver, f'Nhà hàng {restaurant_string}')
    except Exception:
        link = ''
    driver.quit()
    return restaurant_id, link

with open(output_path, mode='w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['restaurant_id', 'restaurant_link'])

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(crawl_link, row) for _, row in df.iterrows()]
        for future in futures:
            restaurant_id, link = future.result()
            writer.writerow([restaurant_id, link])

In [6]:
driver = webdriver.Chrome(options=chrome_options)
actions = ActionChains(driver)
driver.get(link)

In [11]:
exclude_tags = {'tất cả', 'mới nhất', 'all', 'latest', 'video'}
tag_names = [elem.text for elem in elems if elem.text.strip().lower() not in exclude_tags and elem.text.strip()]

In [12]:
tag_names

['Thực đơn',
 'Thực phẩm và đồ uống',
 'Bầu không khí',
 'Nước ép',
 'Cốc tai',
 'Của chủ sở hữu']